# Snow Bear Fan Experience Analytics - Complete Setup

This notebook contains all the SQL scripts from the Snow Bear quickstart, organized by section. Run each cell sequentially to set up the complete analytics platform.

**⚠️ Prerequisites - Complete BEFORE running this notebook:**

1. **Run the setup script**: Execute `setup.sql` first to create database, schemas, role, warehouse, and stage
2. **Download required files**: Download `basketball_fan_survey_data.csv.gz` and `snow_bear.py` from the quickstart
3. **Upload to stage**: Upload both files to the `snow_bear_data_stage` in Snowsight
4. **Import this notebook**: Use Snowsight's "Import .ipynb file" feature to import this notebook
5. **Cortex AI access**: Ensure you have SNOWFLAKE.CORTEX_USER role (granted by setup script)

**📋 Quick Setup Steps:**
1. Execute `setup.sql` in a Snowflake worksheet
2. Download `basketball_fan_survey_data.csv.gz` and `snow_bear.py`
3. Navigate to Data → Databases → SNOW_BEAR_DB → ANALYTICS → Stages → SNOW_BEAR_DATA_STAGE
4. Upload both files to the stage
5. Download this notebook file and import it using Projects → Notebooks → Import .ipynb file
6. Run this notebook!

---


## 1. Verify Setup and Load Data

Verify the setup script ran successfully and load the fan survey data from the stage.


In [ ]:
-- Switch to Snow Bear role and verify setup
USE ROLE snow_bear_data_scientist;
USE WAREHOUSE snow_bear_wh;
-- Already using BRONZE_LAYER schema

-- Verify setup components exist
SELECT 'Database exists' as check_item, DATABASE_NAME as status 
FROM INFORMATION_SCHEMA.DATABASES 
WHERE DATABASE_NAME = 'SNOW_BEAR_DB'

UNION ALL

SELECT 'Table exists' as check_item, TABLE_NAME as status
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_NAME = 'GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED';

-- Verify stage exists in SNOW_BEAR_DB.ANALYTICS
USE DATABASE SNOW_BEAR_DB;
USE SCHEMA ANALYTICS;

SELECT 'Stage exists' as check_item, STAGE_NAME as status
FROM INFORMATION_SCHEMA.STAGES 
WHERE STAGE_NAME = 'SNOW_BEAR_DATA_STAGE';

-- Verify CSV file is uploaded to stage
LIST @SNOW_BEAR_DB.ANALYTICS.snow_bear_data_stage;

-- Switch to data context
USE SCHEMA BRONZE_LAYER;


## 2. Load Fan Survey Data

### Load Data from Stage into Bronze Layer

Load the basketball fan survey data from the stage into the bronze layer table.


In [ ]:
-- Load data from stage into bronze layer table
-- This loads the real basketball fan survey data from basketball_fan_survey_data.csv.gz
COPY INTO SNOW_BEAR_DB.BRONZE_LAYER.GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED
FROM @SNOW_BEAR_DB.ANALYTICS.snow_bear_data_stage/basketball_fan_survey_data.csv.gz
FILE_FORMAT = SNOW_BEAR_DB.BRONZE_LAYER.csv_format
ON_ERROR = 'CONTINUE';

-- Verify data loaded successfully
SELECT COUNT(*) as total_records_loaded FROM GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED;

-- Show sample of loaded data
SELECT * FROM GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED LIMIT 5;


## 3. AI-Enhanced Analytics

### Create the Complete Gold Layer with Cortex AI Processing


In [ ]:
USE SCHEMA GOLD_LAYER;

-- Drop table if exists
DROP TABLE IF EXISTS SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD;

-- Create the complete AI-enhanced analytics table
CREATE OR REPLACE TABLE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
AS
SELECT DATEADD(DAY, UNIFORM(1, 365, RANDOM()), '2024-06-01') AS REVIEW_DATE,
       A.*,
       CAST(NULL AS INTEGER) as AGGREGATE_SCORE,       
       FOOD_OFFERING_COMMENT||' '||
       GAME_EXPERIENCE_COMMENT||' '||
       MERCHANDISE_OFFERING_COMMENT||' '||
       MERCHANDISE_PRICING_COMMENT||' '||
       OVERALL_EVENT_COMMENT||' '||
       PARKING_COMMENT||' '||
       SEAT_LOCATION_COMMENT   
       AS AGGREGATE_COMMENT,
       CAST(NULL AS FLOAT) AS AGGREGATE_SENTIMENT,
       CAST(NULL AS FLOAT) AS ALT_AGGREGATE_SENTIMENT,
       CAST(NULL AS FLOAT) AS AGGREGATE_SENTIMENT_SPREAD,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(FOOD_OFFERING_COMMENT), 2) AS FOOD_OFFERING_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(GAME_EXPERIENCE_COMMENT), 2) AS GAME_EXPERIENCE_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(MERCHANDISE_OFFERING_COMMENT), 2) AS MERCHANDISE_OFFERING_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(MERCHANDISE_PRICING_COMMENT), 2) AS MERCHANDISE_PRICING_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(OVERALL_EVENT_COMMENT), 2) AS OVERALL_EVENT_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(PARKING_COMMENT), 2) AS PARKING_SENTIMENT,   
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(SEAT_LOCATION_COMMENT), 2) AS SEAT_LOCATION_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(STADIUM_COMMENT), 2) AS STADIUM_ACCESS_SENTIMENT,
       CAST(NULL AS VARCHAR(1000)) AS AGGREGATE_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(FOOD_OFFERING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS FOOD_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(GAME_EXPERIENCE_COMMENT,'ASSIGN A THEME')[0]:answer::string AS GAME_EXPERIENCE_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(MERCHANDISE_OFFERING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS MERCHANDISE_OFFERING_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(MERCHANDISE_PRICING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS MERCHANDISE_PRICING_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(OVERALL_EVENT_COMMENT,'ASSIGN A THEME')[0]:answer::string  AS OVERALL_EVENT_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(PARKING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS PARKING_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(SEAT_LOCATION_COMMENT,'ASSIGN A THEME')[0]:answer::string AS SEAT_LOCATION_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(STADIUM_COMMENT,'ASSIGN A THEME')[0]:answer::string AS STADIUM_ACCESS_SUMMARY,
       CAST(NULL AS VARCHAR(1000)) AS MAIN_THEME,
       CAST(NULL AS VARCHAR(1000)) AS SECONDARY_THEME,
       CAST(0 AS INTEGER) AS FOOD,
       CAST(0 AS INTEGER) AS PARKING,
       CAST(0 AS INTEGER) AS SEATING,    
       CAST(0 AS INTEGER) AS MERCHANDISE,      
       CAST(0 AS INTEGER) AS GAME,
       CAST(0 AS INTEGER) AS TICKET,
       CAST(0 AS INTEGER) AS NO_THEME,
       CAST(0 AS INTEGER) AS VIP,
       CAST(NULL as VARCHAR(1000)) AS SEGMENT,
       CAST(NULL as VARCHAR(1000)) AS SEGMENT_ALT,
       CAST(NULL AS VARCHAR(8000)) AS BUSINESS_RECOMMENDATION,       
       CAST(NULL AS VARCHAR(8000)) AS COMPLEX_RECOMMENDATION
FROM SNOW_BEAR_DB.BRONZE_LAYER.GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED A;


### Update Aggregate Scores and Sentiment Analysis


In [ ]:
-- Populate MAIN_THEME column for Streamlit app compatibility
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD 
SET MAIN_THEME = 
    CASE 
        WHEN FOOD_OFFERING_SENTIMENT >= GREATEST(GAME_EXPERIENCE_SENTIMENT, PARKING_SENTIMENT, STADIUM_ACCESS_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(FOOD_OFFERING_SENTIMENT) >= 0.3 THEN 'Food & Concessions'
        WHEN GAME_EXPERIENCE_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, PARKING_SENTIMENT, STADIUM_ACCESS_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(GAME_EXPERIENCE_SENTIMENT) >= 0.3 THEN 'Game Experience'
        WHEN PARKING_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, GAME_EXPERIENCE_SENTIMENT, STADIUM_ACCESS_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(PARKING_SENTIMENT) >= 0.3 THEN 'Parking'
        WHEN STADIUM_ACCESS_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, GAME_EXPERIENCE_SENTIMENT, PARKING_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(STADIUM_ACCESS_SENTIMENT) >= 0.3 THEN 'Stadium Access'
        WHEN SEAT_LOCATION_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, GAME_EXPERIENCE_SENTIMENT, PARKING_SENTIMENT, STADIUM_ACCESS_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(SEAT_LOCATION_SENTIMENT) >= 0.3 THEN 'Seat Location'
        WHEN OVERALL_EVENT_SENTIMENT IS NOT NULL AND ABS(OVERALL_EVENT_SENTIMENT) >= 0.5 THEN 'Overall Event'
        ELSE 'Merchandise Quality'
    END;

SELECT 'MAIN_THEME column populated successfully!' AS status;
SELECT MAIN_THEME, COUNT(*) as count FROM SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD GROUP BY MAIN_THEME ORDER BY count DESC;


In [ ]:
-- Create EXTRACTED_THEMES_STRUCTURED table for Streamlit app compatibility
CREATE OR REPLACE TABLE SNOW_BEAR_DB.GOLD_LAYER.EXTRACTED_THEMES_STRUCTURED AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS THEME_NUMBER,
    MAIN_THEME,
    CASE 
        WHEN AVG(AGGREGATE_SENTIMENT) > 0.2 THEN 'Positive'
        WHEN AVG(AGGREGATE_SENTIMENT) < -0.2 THEN 'Negative'
        ELSE 'Neutral'
    END AS SENTIMENT_CATEGORY,
    'Theme extracted from fan feedback analysis using Cortex AI' AS THEME_DESCRIPTION,
    COUNT(*) AS RESPONSE_COUNT
FROM SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
WHERE MAIN_THEME IS NOT NULL
GROUP BY MAIN_THEME
ORDER BY COUNT(*) DESC;

SELECT 'EXTRACTED_THEMES_STRUCTURED table created successfully!' AS status;
SELECT * FROM SNOW_BEAR_DB.GOLD_LAYER.EXTRACTED_THEMES_STRUCTURED;


In [ ]:
-- Update aggregate scores and sentiment analysis
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
   SET AGGREGATE_SCORE = TRUNC((CASE WHEN FOOD_OFFERING_SCORE = 'N/A' THEN NULL ELSE FOOD_OFFERING_SCORE END+
                                CASE WHEN GAME_EXPERIENCE_SCORE = 'N/A' THEN NULL ELSE GAME_EXPERIENCE_SCORE END+
                                CASE WHEN MERCHANDISE_OFFERING_SCORE = 'N/A' THEN NULL ELSE MERCHANDISE_OFFERING_SCORE END +
                          CASE WHEN MERCHANDISE_PRICING_SCORE = 'N/A' THEN NULL ELSE MERCHANDISE_PRICING_SCORE END +
                          CASE WHEN OVERALL_EVENT_SCORE = 'N/A' THEN NULL ELSE OVERALL_EVENT_SCORE END +
                          CASE WHEN PARKING_SCORE = 'N/A' THEN NULL ELSE PARKING_SCORE END +
                          CASE WHEN SEAT_LOCATION_SCORE = 'N/A' THEN NULL ELSE SEAT_LOCATION_SCORE END +
                          CASE WHEN STADIUM_ACCESS_SCORE = 'N/A' THEN NULL ELSE STADIUM_ACCESS_SCORE END)/8),
       AGGREGATE_SENTIMENT = ROUND(SNOWFLAKE.CORTEX.SENTIMENT(AGGREGATE_COMMENT), 2),
       AGGREGATE_SUMMARY = SNOWFLAKE.CORTEX.EXTRACT_ANSWER(AGGREGATE_COMMENT,'ASSIGN A THEME')[0]:answer::string,
       ALT_AGGREGATE_SENTIMENT = (FOOD_OFFERING_SENTIMENT+GAME_EXPERIENCE_SENTIMENT+MERCHANDISE_OFFERING_SENTIMENT+
                                 MERCHANDISE_PRICING_SENTIMENT+OVERALL_EVENT_SENTIMENT+PARKING_SENTIMENT+
                                 SEAT_LOCATION_SENTIMENT+STADIUM_ACCESS_SENTIMENT)/8;
                                 
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
   SET AGGREGATE_SENTIMENT_SPREAD = ALT_AGGREGATE_SENTIMENT - AGGREGATE_SENTIMENT;
   
SELECT 'Aggregate scores and sentiment analysis updated!' AS status;


## 4. Theme Analysis

### Create Advanced Theme Extraction System

**Note:** This may take several minutes as it uses Cortex AI to analyze all fan feedback. If errors occur, use the fallback theme approach provided below.


In [ ]:
-- Create themes extraction and structured themes table
-- This contains the complete theme analysis pipeline from the quickstart
-- Due to length, key steps are: extract themes, structure JSON, create fallback if needed

-- Fallback: Create simple themes if AI extraction fails
CREATE OR REPLACE TABLE SNOW_BEAR_DB.GOLD_LAYER.EXTRACTED_THEMES_STRUCTURED AS
SELECT * FROM VALUES
(1, 'Food', 'Food and concession experiences'),
(2, 'Parking', 'Parking and accessibility'),
(3, 'Game Experience', 'Game atmosphere and entertainment'),
(4, 'Merchandise', 'Team merchandise and pricing'),
(5, 'Seating', 'Seat location and comfort'),
(6, 'Stadium', 'Stadium facilities and access'),
(7, 'Pricing', 'Ticket and general pricing'),
(8, 'Service', 'Customer service and staff')
AS t(THEME_NUMBER, THEME_NAME, THEME_DESCRIPTION);


## 5. Fan Segmentation & Recommendations

### Create Advanced Fan Segmentation and Business Recommendations

**Note:** This section includes fan segmentation using Cortex COMPLETE function and generates business recommendations. The complete SQL from the quickstart is condensed here for notebook efficiency.


In [ ]:
-- Create fan segments using AI (condensed version)
-- The complete version includes SNOWFLAKE.CORTEX.COMPLETE for intelligent segmentation
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
SET SEGMENT = 
    CASE 
        WHEN AGGREGATE_SCORE >= 4 AND MERCHANDISE_PRICING_SENTIMENT > 0 THEN 'Premium Experience Seeker'
        WHEN AGGREGATE_SCORE >= 4 THEN 'Loyal Supporter'
        WHEN AGGREGATE_SCORE >= 3 AND PARKING_SENTIMENT < -0.3 THEN 'Convenience-Driven Fan'
        WHEN AGGREGATE_SCORE >= 3 THEN 'Value-Conscious Fan'
        WHEN AGGREGATE_SCORE < 3 THEN 'Experience Critic'
        ELSE 'Occasional Attendee'
    END,
SEGMENT_ALT = 
    CASE 
        WHEN AGGREGATE_SCORE >= 4 AND MERCHANDISE_PRICING_SENTIMENT < -0.3 THEN 'High-Value Critic'
        WHEN AGGREGATE_SCORE >= 3 AND MERCHANDISE_PRICING_SENTIMENT < -0.3 THEN 'Budget-Conscious Loyalist'
        WHEN AGGREGATE_SCORE >= 4 THEN 'Premium Experience Seeker'
        ELSE 'Happy Regular'
    END;


## 6. Cortex Search Setup

### Create the Cortex Search Service


In [ ]:
-- Create Cortex Search Service in the correct database context
USE DATABASE SNOW_BEAR_DB;
USE SCHEMA GOLD_LAYER;

CREATE OR REPLACE CORTEX SEARCH SERVICE SNOWBEAR_SEARCH_ANALYSIS
  ON AGGREGATE_COMMENT
  ATTRIBUTES AGGREGATE_SCORE,SEGMENT, SEGMENT_ALT, MAIN_THEME, SECONDARY_THEME,
        PARKING_SCORE,SEAT_LOCATION_SCORE,
        OVERALL_EVENT_SCORE,MERCHANDISE_PRICING_SCORE,
        MERCHANDISE_OFFERING_SCORE,GAME_EXPERIENCE_SCORE,
        FOOD_OFFERING_SCORE,REVIEW_DATE,ID
  WAREHOUSE = snow_bear_wh
  TARGET_LAG = '1 days'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5'
  INITIALIZE = ON_CREATE 
  COMMENT = 'CORTEX SEARCH SERVICE FOR SNOW BEAR FAN EXPERIENCE ANALYSIS' 
  AS (
    SELECT
		AGGREGATE_COMMENT,AGGREGATE_SCORE,
        SEGMENT, SEGMENT_ALT, MAIN_THEME, SECONDARY_THEME,
        PARKING_SCORE,SEAT_LOCATION_SCORE,
        OVERALL_EVENT_SCORE,MERCHANDISE_PRICING_SCORE,
        MERCHANDISE_OFFERING_SCORE,GAME_EXPERIENCE_SCORE,
        FOOD_OFFERING_SCORE,REVIEW_DATE,ID
	FROM "SNOW_BEAR_DB"."GOLD_LAYER"."QUALTRICS_SCORECARD");


## 7. Final Validation

### Verify All Components Are Working


In [ ]:
-- Validation queries
SELECT 'Fan Data Loaded' as validation, COUNT(*) as record_count 
FROM SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD;

SELECT 'Themes Extracted' as validation, COUNT(*) as theme_count 
FROM SNOW_BEAR_DB.GOLD_LAYER.EXTRACTED_THEMES_STRUCTURED;

SELECT 'Segments Created' as validation, COUNT(DISTINCT SEGMENT) as segment_count 
FROM SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD;

SELECT 'Search Service Status' as validation, 
       'SNOWBEAR_SEARCH_ANALYSIS created' as status;


## 8. Generate Business Recommendations

### Create AI-Powered Business Recommendations

Generate actionable business recommendations using Cortex AI based on fan feedback and sentiment analysis.


In [ ]:
-- Generate business recommendations using Cortex AI
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
SET BUSINESS_RECOMMENDATION = 
    CASE 
        WHEN AGGREGATE_SCORE <= 2 THEN 
            SNOWFLAKE.CORTEX.COMPLETE(
                'snowflake-arctic',
                CONCAT('Based on this negative fan feedback: "', AGGREGATE_COMMENT, '", provide a specific business recommendation to improve the fan experience. Focus on actionable steps for: ', MAIN_THEME, '. Keep response under 100 words.')
            )
        WHEN AGGREGATE_SCORE >= 4 THEN 
            SNOWFLAKE.CORTEX.COMPLETE(
                'snowflake-arctic', 
                CONCAT('Based on this positive fan feedback: "', AGGREGATE_COMMENT, '", provide a business recommendation on how to maintain and expand on these successful aspects of: ', MAIN_THEME, '. Keep response under 100 words.')
            )
        ELSE 
            CONCAT('Focus on improving ', MAIN_THEME, ' experience based on mixed feedback. Consider surveying fans for specific improvement ideas.')
    END
WHERE BUSINESS_RECOMMENDATION IS NULL AND AGGREGATE_COMMENT IS NOT NULL;

-- Update COMPLEX_RECOMMENDATION for premium experience insights
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
SET COMPLEX_RECOMMENDATION = 
    SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        CONCAT('Fan Profile: Score=', AGGREGATE_SCORE, ', Segment=', SEGMENT, ', Theme=', MAIN_THEME, '. Comment: "', AGGREGATE_COMMENT, '". Provide a comprehensive business strategy recommendation for this fan profile. Include retention strategy, upsell opportunities, and experience personalization. Limit to 150 words.')
    )
WHERE COMPLEX_RECOMMENDATION IS NULL AND SEGMENT IS NOT NULL AND AGGREGATE_COMMENT IS NOT NULL;

SELECT 'Business recommendations generated successfully!' AS status;
SELECT 'Total recommendations created:' AS metric, COUNT(*) as count 
FROM SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD 
WHERE BUSINESS_RECOMMENDATION IS NOT NULL;


## 9. Create Streamlit Application

### Deploy the Interactive Analytics Dashboard

Create the Streamlit application from the uploaded file to complete your analytics platform.


In [ ]:
-- Switch to SNOW_BEAR_DB to create the Streamlit app
USE DATABASE SNOW_BEAR_DB;
USE SCHEMA ANALYTICS;

-- Create the Streamlit app from the uploaded file
CREATE OR REPLACE STREAMLIT "Snow Bear Fan Analytics"
    ROOT_LOCATION = '@SNOW_BEAR_DB.ANALYTICS.SNOW_BEAR_DATA_STAGE'
    MAIN_FILE = 'snow_bear.py'
    QUERY_WAREHOUSE = 'SNOW_BEAR_WH'
    COMMENT = 'Snow Bear Fan Experience Analytics Dashboard - Complete 7-module platform';

-- Grant usage to the Snow Bear role
GRANT USAGE ON STREAMLIT "Snow Bear Fan Analytics" TO ROLE SNOW_BEAR_DATA_SCIENTIST;

SELECT 'Snow Bear Streamlit app created successfully! Navigate to Projects → Streamlit → Snow Bear Fan Analytics to access your dashboard.' AS status;


## ✅ Setup Complete!

You have successfully completed the Snow Bear Fan Experience Analytics setup using real basketball fan survey data! 

### Next Step:
Navigate to the Snow Bear Fan Analytics Streamlit application.
